# 14. Auditoria: Classificação Automática de Riscos

Um dos passos da auditoria é classificar os apontamentos conforme seu risco (Alto, Médio, Baixo) para priorizar correções. LLMs são excelentes classificadores zero-shot.

**Objetivo:** Classificar descrições de falhas de controle e gerar uma justificativa para a nota.

In [ ]:
!pip install -qU langchain langchain-openai langchain-community

In [ ]:
import os
from google.colab import userdata
import getpass

try:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
except:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Digite sua OpenAI API Key: ")

## 1. Definindo a Matriz de Riscos no Prompt

Instruímos o modelo sobre o que constitui cada nível de risco.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.pydantic_v1 import BaseModel, Field

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# Schema de Saída
class ClassificacaoRisco(BaseModel):
    nivel: str = Field(description="Nível de risco: 'Alto', 'Médio' ou 'Baixo'")
    justificativa: str = Field(description="Explicação breve do porquê desse nível de risco")
    acao_sugerida: str = Field(description="Ação imediata recomendada")

structured_llm = llm.with_structured_output(ClassificacaoRisco)

sistema = """
Você é um especialista em Gestão de Riscos Corporativos.
Classifique o seguinte apontamento de auditoria interna conforme a matriz:

- ALTO: Perda financeira significativa (> R$ 100k), fraude, violação legal grave (LGPD, Anticorrupção) ou risco de imagem.
- MÉDIO: Falha de processo repetitiva, perda financeira moderada (< R$ 100k) ou dados imprecisos.
- BAIXO: Erros pontuais, documentação faltante não crítica ou melhoria de eficiência.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", sistema),
    ("human", "Apontamento: {apontamento}")
]) | structured_llm

## 2. Testando com Casos Reais

Vamos passar alguns cenários para ver a classificação.

In [ ]:
cenarios = [
    "O sistema de almoxarifado permite saída de mercadoria sem requisição aprovada. Identificada perda de estoque de R$ 500.000 no ano.",
    "Três relatórios de despesas de viagem de Junho/2023 estavam sem carimbo da recepção, mas com notas fiscais válidas.",
    "Identificamos um funcionário do Depto de Compras que é sócio de um fornecedor recém-contratado sem declaração de conflito de interesses."
]

for cenario in cenarios:
    print(f"--- CENÁRIO: {cenario[:60]}... ---")
    res = prompt.invoke({"apontamento": cenario})
    print(f"NÍVEL: {res.nivel}")
    print(f"JUSTIFICATIVA: {res.justificativa}")
    print(f"AÇÃO: {res.acao_sugerida}\n")

## 3. Tagging Chain (Opção Alternativa)

O LangChain também possui `create_tagging_chain` para casos simples onde queremos apenas categorizar (tags).

## Conclusão

Automatizar a classificação inicial ajuda a direcionar o foco dos auditores seniores para os problemas de Risco Alto.